In [1]:
# Load data
import pandas as pd

DIFFERENCE_THRESHS = [1.05, 1.1, 1.5, 3, 4.5]

data = pd.read_csv("ae_trials.csv")

data["leader"] = data["leader"].astype(float)
data["name"] = data["name"].astype(float)

data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'ae_trials.csv'

In [2]:
# Get the lowest average value for val_loss by for input_size
from math import isnan
from statistics import mean

input_size_results = []
for input_size in [4096]:
    aes = data[data["input_size"] == input_size]
    aes = aes[["name", "type", "leader", "val_loss"]].to_dict("records")
    results = {}
    for ae in aes:
        if ae["type"] == "Vanilla AE" and not isnan(ae["leader"]):
            results[str(int(ae["leader"]))] = results.get(str(int(ae["leader"])), []) + [ae["val_loss"]]
        elif ae["type"] == "Adversarial AE":
            results[str(int(ae["name"]))] = results.get(str(int(ae["name"])), []) + [ae["val_loss"]]
    
    new_res = {}
    print(results)
    for key, value in results.items():
        if len(value) == 2:
            new_res[key] = mean(value)

    minimum = min(new_res.values())
    for key, value in new_res.items():
        if value == minimum:
            input_size_results.append([key, value])
            break
    del new_res[key]
    
    minimum = min(new_res.values())
    for key, value in new_res.items():
        if value == minimum:
            input_size_results.append([key, value])
            break
    del new_res[key]
            
    minimum = min(new_res.values())
    for key, value in new_res.items():
        if value == minimum:
            input_size_results.append([key, value])
            break
    del new_res[key]
              
print(input_size_results)

NameError: name 'data' is not defined

In [3]:
ae_reses = []

for ae in input_size_results:
    adversarial = data[data["name"] == float(ae[0])]
    ae_res = adversarial[["dataset", "input_size", "z_layer_divisor", "hidden_layers_divisor", "hidden_layers", "dropout", "learning_rate"]].to_dict("records")[0]
    ae_res["aa_name"] = str(ae[0])
    ae_res["Dataset"] = ae_res["dataset"]
    ae_res["Input Size"] = str(int(ae_res["input_size"]))
    ae_res["Z Layer Size"] = str(int(ae_res["input_size"]/ae_res["z_layer_divisor"]))
    ae_res["Hidden Layer Size(s)"] = str(int(ae_res["input_size"]/ae_res["hidden_layers_divisor"]))
    if ae_res["hidden_layers"] > 1:
        ae_res["Hidden Layer Size(s)"] += ", " + str(int(ae_res["input_size"]/(2 * ae_res["hidden_layers_divisor"])))
        
    ae_res["Number of Hidden Layers"] = ae_res["hidden_layers"]
    
    vanilla = data[data["leader"] == float(ae[0])]
    ae_res["van_name"] = vanilla["name"].values.tolist()[0]
    ae_res["Vanilla Reconstruction Loss"] = vanilla["val_loss"].values.tolist()[0]
    ae_res["Adversarial Reconstruction Loss"] = adversarial["val_loss"].values.tolist()[0]
    
    ae_reses.append(ae_res)

ae_reses = pd.DataFrame(ae_reses)
ae_reses[["Dataset", "Input Size", "Z Layer Size", "Number of Hidden Layers", "Hidden Layer Size(s)", "Z Layer Size", "Vanilla Reconstruction Loss", "Adversarial Reconstruction Loss"]].to_csv("ae_table.csv")

In [4]:
# load ll classifiers
ll_data = pd.read_csv("latent_trials.csv")
ll_data["Based on AE"] = ll_data["Based on AE"].astype(float)

for ae in ae_reses.to_dict("records"):
    print(ae["input_size"])
    
    print(ll_data[ll_data["Based on AE"] == float(ae["aa_name"])].groupby("Classifier").mean("f-score").sort_values("f-score")[["precision", "recall", "f-score", "auc"]])
    print(ll_data[ll_data["Based on AE"] == float(ae["van_name"])].groupby("Classifier").mean("f-score").sort_values("f-score")[["precision", "recall", "f-score", "auc"]])
    print()
    

4096
                            precision    recall   f-score       auc
Classifier                                                         
Discriminator                0.000000  0.000000  0.000000       NaN
Discriminator_Thresh_1       0.000000  0.000000  0.000000       NaN
Discriminator_Thresh_10      0.000000  0.000000  0.000000       NaN
Discriminator_Thresh_20      0.000000  0.000000  0.000000       NaN
Discriminator_Thresh_30      0.000000  0.000000  0.000000       NaN
Discriminator_Thresh_40      0.000000  0.000000  0.000000       NaN
Discriminator_Thresh_5       0.000000  0.000000  0.000000       NaN
Discriminator_Thresh_50      1.000000  0.450000  0.618182       NaN
ReconstructionThreshold      0.870884  1.000000  0.927653       NaN
XGBClassifier                0.985294  0.984375  0.984360  0.982572
ExtraTreesClassifier         0.985294  1.000000  0.992424  1.000000
GradientBoostingClassifier   0.985294  1.000000  0.992424  1.000000
LogisticRegression           0.985294  1.00

In [5]:
# Calculate best classifiers
import numpy as np

ll_data["type"] = [data[data["name"] == based_on]["type"].values[0] for based_on in ll_data["Based on AE"].values.tolist()]
ll_data["dataset"] = [data[data["name"] == based_on]["dataset"].values[0] for based_on in ll_data["Based on AE"].values.tolist()]
ll_data["input_size"] = [data[data["name"] == based_on]["input_size"].values[0] for based_on in ll_data["Based on AE"].values.tolist()]

ll_data[["input_size", "type", "dataset", "Classifier", "precision", "recall", "f-score", "auc"]].head()

final_ll_data = []
for name, group in ll_data.groupby(["input_size", "type", "Classifier", "Based on AE"]):
    group = group.to_dict("list")
    data_group = {"Input Size": str(group["input_size"][0]), "Type": str(group["type"][0]), "Classifier": str(group["Classifier"][0])}
    data_group["F-score"] = np.mean(group["f-score"])
    data_group["F-score Standard Deviation"] = np.std(group["f-score"])
    data_group["Precision"] = np.mean(group["precision"])
    data_group["Precision Standard Deviation"] = np.std(group["precision"])
    data_group["Recall"] = np.mean(group["recall"])
    data_group["Recall Standard Deviation"] = np.std(group["recall"])
    final_ll_data.append(data_group)

final_ll_data = pd.DataFrame(final_ll_data)
final_ll_data_duplicate = final_ll_data
final_ll_data = final_ll_data.groupby(["Input Size", "Type", "Classifier"]).max(["F-score"])[["Precision", "Recall", "F-score", "Precision Standard Deviation", "Recall Standard Deviation", "F-score Standard Deviation"]]
final_ll_data["calculated_values"] = ["%.3f σ %.2f (%.3f σ %.2f, %.3f σ %.2f)" % (x[0], x[1], x[2], x[3], x[4], x[5]) for x in final_ll_data[["F-score", "F-score Standard Deviation", "Precision", "Precision Standard Deviation", "Recall", "Recall Standard Deviation"]].values.tolist()]
final_ll_data.to_csv("ll_with_ae_details.csv")